In [1]:
import tensorflow as tf
import os
import json
import librosa
import numpy as np
DATA_PATH = 'data.json'
MODEL_PATH = 'model.h5'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


2023-08-20 19:32:55.460353: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 19:32:56.558968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
print(os.environ.get('TF_FORCE_GPU_ALLOW_GROWTH'))

true


In [3]:
print(os.environ.get('CUDA_HOME'))


/usr/local/cuda


In [4]:
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


In [5]:
# open
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

In [6]:
data['mappings'][0]

'two'

In [7]:
class _Keyword_Spotting_Serveice:
    model = None
    _mapping = data['mappings'].copy()
    _instance = None

    def predict(self, file_path):
        # extract MFCCs
        MFCCs = self.preprocess(file_path)

        # convert 2d MFCCs array into 4d array
        MFCCs = MFCCs[np.newaxis, ..., np.newaxis]

        # make prediction
        predictions = self.model.predict(MFCCs)
        predicted_index = np.argmax(predictions)
        predicted_keyword = self._mapping[predicted_index]
        return predicted_keyword
    
    def preprocess(self, file_path, n_mfcc=13, n_fft=2048, hop_length=512):

        # load audio file
        signal, sr = librosa.load(file_path)
        # ensure consistency in the audio file length
        if len(signal) >= 22050:
            signal = signal[:22050]

            # extract MFCCs
            MFCCs = librosa.feature.mfcc(y=signal, n_mfcc=n_mfcc, 
                                         n_fft=n_fft, hop_length=hop_length)
            
            return MFCCs.T
    

In [9]:
def Keyword_Spotting_Serveice():

    # ensure that we only have 1 instance of KSS
    if _Keyword_Spotting_Serveice._instance is None:
        _Keyword_Spotting_Serveice._instance = _Keyword_Spotting_Serveice()
        _Keyword_Spotting_Serveice.model     = tf.keras.models.load_model(MODEL_PATH)

    return _Keyword_Spotting_Serveice._instance



if __name__ == "__main__":
    kss = Keyword_Spotting_Serveice()
    labels = kss.predict('/home/long/Source-Code/SpechCommandAppication/archive/augmented_dataset/augmented_dataset/tree/2.wav')

1/1 [==============================] - 0s 38ms/step


In [10]:
print(labels)

tree
